# Майнор "Прикладные задачи анализа данных"
## Домашнее задание 1 

Задание выполнено в группе: Дымченко Софья, Рогачевская Анастасия, Синицин Антон [ИАД3]

## Постановка задачи 
В данной работе предлагается реализовать генератор описания прогноза погоды по трем шаблонам на ближайшие 10 дней в выбранном городе (Уренгой) выгрузив данные с сайта gismeteo.ru:
https://www.gismeteo.ru/weather-urengoy-3972/10-days/

### Формулировка задачи

(1) Собрать данные с сайта и вывести их в виде таблицы, в столбцах которой даты и дни недели, а в строках следующие показатели:

* минимальная температура,
* максимальная температура,
* скорость ветра,
* уровень осадков.

(2) Написать генератор описания прогноза, состоящий из трех предлагаемых шаблонов. На входе от пользователя str(day1, day2) - две даты, за которые пользователь хочет узнать как изменилась погода. На выходе текст из трех предложений - изменение температуры, скорости ветра и уровня осадков.

(3) Ответить на вопросы.

### 1. Сбор данных [3 балла]



Будем использовать библиотеку BeautifulSoup4 для парсинга веб страницы, выгруженной с помощью urllib.request.

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import numpy as np

page = urlopen("https://www.gismeteo.ru/weather-urengoy-3972/10-days/")
soup = BeautifulSoup(page, 'lxml')


* В инспекторе в браузере на странице ищем теги, внутри которых хранится нужная нам информация о показателях и датах. 
* В основном это div-ы с атрибутом class, названия которого нам и нужны, так как они более или менее уникальные. 
* Для каждого показателя с помощью bs4 находим теги, извлекаем текст, отрезаем пробелы и \n, заносим в списки.

Создадим функцию, так как для каждого показателя последнее действие отличается только переменной списка, куда класть значения, и строкой названия класса нужного тега.

In [2]:
dates, weekdays, maxtemps, mintemps, winds, precips = [], [], [], [], [], []

def indicator(class_name_string, ind_list):
    for pos in soup.findAll(class_=re.compile(class_name_string))[:10]:
        ind_list.append(pos.get_text().strip())
    return ind_list


Теперь заполним списки данными, а так же немного подправим их типы:

In [3]:
indicator("date__date", dates)
dates[0]=re.match('\d\d?', dates[0]).group()
dates = [int(x) for x in dates]
dates

[17, 18, 19, 20, 21, 22, 23, 24, 25, 26]

In [4]:
indicator("date__day", weekdays)
weekdays = [x.lower() for x in weekdays]
weekdays

['сб', 'вс', 'пн', 'вт', 'ср', 'чт', 'пт', 'сб', 'вс', 'пн']

In [5]:
indicator("maxt", maxtemps)
for i in range(len(maxtemps)):
    if '−' in maxtemps[i]:
        maxtemps[i] = -int(maxtemps[i][1:])
    else:
        maxtemps[i] = int(maxtemps[i])
maxtemps

[-11, -13, -14, -20, -26, -26, -29, -29, -28, -26]

In [6]:
indicator("mint", mintemps)
for i in range(len(mintemps)):
    if '−' in mintemps[i]:
        mintemps[i] = -int(mintemps[i][1:])
    else:
        mintemps[i] = int(mintemps[i])
mintemps

[-16, -17, -18, -27, -30, -29, -31, -31, -32, -33]

In [7]:
indicator("w_wind__warning", winds)
winds = [int(x) for x in winds]
winds

[5, 7, 7, 7, 4, 7, 7, 8, 8, 14]

In [8]:
indicator("precipitation__value", precips)  
for i in range(len(maxtemps)):
    if ',' in precips[i]:
        precips[i] = precips[i].replace(',', '.')
        precips[i] = float(precips[i])
    else:
        precips[i] = float(precips[i])
precips

[0.6, 0.5, 0.6, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 1.2]

Сформируем окончательно словарь dataframe для удобства:

In [9]:
days = []
for i in range(len(dates)):
    days.append(str(dates[i]) + " " + weekdays[i])

line_names = ['максимальная температура', 'минимальная температура', 'скорость ветра', 'уровень осадков']

In [10]:
ind=zip(maxtemps, mintemps, winds, precips)
data_forecast = dict(zip(days, ind))

In [11]:
import pandas as pd
data = pd.DataFrame(data_forecast, index=line_names)

In [12]:
data


,17 сб,18 вс,19 пн,20 вт,21 ср,22 чт,23 пт,24 сб,25 вс,26 пн
максимальная температура,-11.0,-13.0,-14.0,-20.0,-26.0,-26.0,-29.0,-29.0,-28.0,-26.0
минимальная температура,-16.0,-17.0,-18.0,-27.0,-30.0,-29.0,-31.0,-31.0,-32.0,-33.0
скорость ветра,5.0,7.0,7.0,7.0,4.0,7.0,7.0,8.0,8.0,14.0
уровень осадков,0.6,0.5,0.6,0.1,0.0,0.0,0.0,0.0,0.0,1.2


In [13]:
ind_2=zip(weekdays, maxtemps, mintemps, winds, precips)
data_forecastV2 = dict(zip(dates, ind_2))

### 2. Генератор описания прогноза погоды [4 балла]


In [14]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()


Из этой библиотеки воспользуемся следующими методами:
    - Parse.inflect({''}) - для приведения слова к необходимой падежной форме
    - Parse.make_agree_with_number() - для согласования слова с числительным

В наших шаблонах имеются предлоги с вариациями: *"с/со"* и *"в/во"*. Правильным было бы написать правила использования вариаций предлогов в зависимости от следующего за ним слова (по первым буквам), но так как наш проект небольшой, а использование предлогов ограничивается только **днями недели**, то было решено реализовать это с помощью словарей (switch case). А также создадим словарь нормальных названий дней недели, так как выгружены были краткие.

In [15]:
var_with = {
    'понедельник': 'с',
    'вторник' : 'со',
    'среда': 'cо',
    'четверг': 'с',  
    'пятница': 'с',
    'суббота': 'с',
    'воскресенье': 'с'
}

var_from = {
    'понедельник': 'в',
    'вторник' : 'во',
    'среда': 'в',
    'четверг': 'в',  
    'пятница': 'в',
    'суббота': 'в',
    'воскресенье': 'в'
}

week_dc = {
    'пн': 'понедельник',
    'вт': 'вторник',
    'ср': 'среда',
    'чт': 'четверг',  
    'пт': 'пятница',
    'сб': 'суббота',
    'вс': 'воскресенье'
}


Предполагаем, что "пользователем" на ввод подаются даты таким образом:
*  просто число - '16',
*  число и месяц - '18.02' (на месте точки может быть любой разделитель или пробел),

и оба дня могут подаваться любым из способов независимо друг от друга.

Напишем функцию, которая верно "ловит" числа, о погоде которых хочет узнать "пользователь", т.к. она нам пригодится несколько раз.

In [16]:
import re

def parse_input(day1, day2):
    '''
    Ищет в вводимых строчках с помощью регулярных выражений
    первые одну или две цифры, являющиеся числом в дате и
    кладет их на месте в те же переменные как int 
    '''
    pattern = re.compile('\d\d?')
    day1 = int(pattern.match(str(day1)).group(0))
    day2 = int(pattern.match(str(day2)).group(0))
    return day1, day2


Напишем функции, к которым интуитивно проще обращаться, для вызова определенного показателя определенного дня.

In [17]:
def week(day):
    return data_forecastV2[day][0]
    # день недели str
    
def temp(day):
    return (data_forecastV2[day][1] + data_forecastV2[day][2])/2
    # средняя температура int

def wind(day):
    return data_forecastV2[day][3]
    # скорость ветра 

def precip(day):
    return data_forecastV2[day][4]
    # уровень осадков
    

Напишем функцию вызова первого шаблона, который описывает изменение температуры:

**В *день2* похолодает/потеплеет на X градус(-а, -ов) по сравнению с *день1*.**

In [18]:
def temp_verb(day1, day2): 
    '''
    Возращает нужный глагол в зависимости
    от изменения средней температуры,
    day2 идет после day1 
    '''
    if temp(day1) > temp(day2):
        return 'похолодает'
    if temp(day1) <= temp(day2):
        return 'потеплеет'
    
def temp_dif(day1, day2):
    ''' Разница температур между днями'''
    return abs(temp(day2)-temp(day1))

def temperature(day1, day2):
    '''
    Возвращает строку шаблона, ставя в правильный падеж день недели и город,
    согласовывая с числительным градусы, вставляя правильные вариации предлогов
    и нужный глагол.

    '''
    day1, day2 = parse_input(day1, day2)
    weekday1=week_dc[week(day1)]
    weekday2=week_dc[week(day2)]

    
    answer = ' '.join([var_from[weekday2].capitalize(), 
                      morph.parse(weekday2)[0].inflect({'accs'}).word, 
                      'в', morph.parse('Уренгой')[0].inflect({'loct'}).word.capitalize(), 
                      temp_verb(day1, day2),
                      'на', str(temp_dif(day1, day2)),
                      morph.parse('градусы')[0].make_agree_with_number(temp_dif(day1, day2)).word, 
                      'по сравнению', var_with[weekday1],  
                      morph.parse(weekday1)[0].inflect({'ablt'}).word]) + '.'
    return answer


Напишем функцию вызова второго шаблона, который описывает изменение скорости ветра:

**Скорость ветра изменится на X единиц в день2 по сравнению с день1.**

In [19]:
def wind_dif(day1, day2):
    '''Разница скорости ветра между днями'''
    return abs(wind(day1) - wind(day2))

def wind_speed(day1, day2):
    '''
    Возвращает строку шаблона, ставя в правильный падеж день недели,
    согласовывая с числительным единицы, вставляя правильные вариации предлогов.
    '''
    day1, day2 = parse_input(day1, day2)
    weekday1=week_dc[week(day1)]
    weekday2=week_dc[week(day2)]
    answer=' '.join(['Скорость ветра изменится на', str(wind_dif(day1, day2)), 
                    morph.parse('единицу')[0].make_agree_with_number(wind_dif(day1, day2)).word, 
                    var_from[weekday1], morph.parse(weekday1)[0].inflect({'accs'}).word, 
                    'по сравнению', var_with[weekday2], 
                    morph.parse(weekday2)[0].inflect({'ablt'}).word]) + '.'
    return answer


Напишем функцию вызова третьего шаблона, который описывает изменение уровня осадков:

**Уровень осадков повысится/понизится на X единиц за Y дней.**

In [20]:
def precip_verb(day1, day2):
    '''
    Возращает нужный глагол в зависимости
    от изменения уровня осадков,
    day2 идет после day1 
    '''
    if precip(day1) > precip(day2):
        return 'понизится'
    if precip(day1) <= precip(day2):
        return 'повысится'

def precip_dif(day1, day2):
    '''Разница уровня осадков между днями'''
    return round(abs(precip(day2) - precip(day1)), 2)
    
def precip_level(day1, day2):
    '''
    Возвращает строку шаблона, ставя в правильный падеж слово "дни",
    согласовывая с числительным единицы и вставляя нужный глагол.
    '''
    day1, day2 = parse_input(day1, day2)
    answer=' '.join(['Уровень осадков', precip_verb(day1, day2), 
                    'на', str(precip_dif(day1, day2)), 
                    morph.parse('единицы')[0].make_agree_with_number(precip_dif(day1, day2)).word, 
                    'за', str(abs(day2 - day1)), 
                    morph.parse('дни')[0].make_agree_with_number((day2 - day1)).word]) + '.'
    return answer


Проверим работоспособность шаблонов:

In [21]:
day1, day2 = '20.02', '26'
print(temperature(day1, day2), wind_speed(day1, day2), precip_level(day1, day2), sep='\n')


В понедельник в Уренгое похолодает на 6.0 градусов по сравнению со вторником.
Скорость ветра изменится на 7 единиц во вторник по сравнению с понедельником.
Уровень осадков повысится на 1.1 единиц за 6 дней.


### 3. Ответьте на вопросы [3 балла]
* **В каких других задачах (помимо описания прогноза погоды) может понадобиться генерировать текст по шаблонам? В каких задачах может понадобиться генерировать текст об изменении числовых показателей по шаблонам?**

1) Генерация текстов по шаблонам может применяться при автоматическом **составлении юридических документов** (договоров, заявлений, отчетов для бухгалтерии и т.д.). Более того, SEO-специалисты часто используют тексты как инструмент продвижения сайтов в поисковых системах, а именнно для **размножения статей**: на основе одного текста создаются его копии (чаще всего путем использования синонимов и схожих конструкций), далее они размещаются на других ресурсах и блогах с целью рекламы. Такие копии должны достаточно различаться друг от друга и казаться "уникальными", но при этом важно сохранить тему статьи и основной смысл. Также можно использовать генерацию текста по шаблонам для **наполнения ресурса контентом**. Например, таким образом можно составлять **описания товаров** для интернет-магазина или кратко оценивать товар предложениями, составленными на основе отзывов покупателей по разным качествам. Кроме этого, такие генераторы могут помочь исследователям для состваления **отчетов об их исследованиях**: вместо того, чтобы каждый раз "от руки" описывать графики, прогресс исследования и другие изменения числовых показателей одного и того же, можно использовать шаблоны.

2) Сгенерированные тексты об изменении числовых показателей часто встречаются в почтовых рассылках и смс-уведомлениях. Можно привести следующие примеры: 
* сообщения от банков с информацией о текущем балансе или совершенных операциях 
* письма от сервисов, уведомляющие о снижении/повышении цен на товары (например, на авиабилеты, которые ранее просматривал пользователь)
* сведения о ситуации на дорогах, пробках 
![Image](https://c.radikal.ru/c01/1802/e2/983c22bc7964.png)

* **Шаблоны, которые вы использовали в этом задании, имеют фиксированную структуру. Фактически, ваша задача заключалась в том, чтобы подставить в шаблон число и согласовать единицы измерения с этим числом или подставить в шаблон название города и согласовать его с предлогом. Как можно разнообразить эти шаблоны? Как знание синтаксической структуры предложения может помочь в этой задаче?**

1) Шаблонам можно придать эмоциональную окраску, добавив междометия, слова или словосочетания, чтобы выразить оценку текущей погоды и сделать текст более "живым". Например, вот так:
* *Ура! Кажется, весна пришла: во вторник в Уренгое потеплеет на 10 градусов по сравнению с субботой.*
* *Одевайтесь теплее! Во вторник в Уренгое похолодает на 10 градусов по сравнению с субботой.*

Кроме того, можно использовать синонимы для глаголов и добавить наречия:
* *Уровень осадков незначительно уменьшится на 0.7 единиц за 3 дня.*

Также, когда числовые показатели не изменяются, то можно заменить вывод нулевой разницы высказыванием:

* *Скорость ветра не изменится во вторник по сравнению cо средой.*

2) Для того, чтобы разнообразить текст, мы добавляем новые слова и увеличиваем сложность фразы. Знание синтаксической структуры предложения помогает сохранить правильный порядок слов и избежать ошибок: например таких, когда слово расположено слишком далеко от того, от которого оно зависит:
* *Незначительно уровень осадков уменьшится на 0.7 единиц за 3 дня.*